In [60]:
import pandas as pd
import numpy as np
import math

### Carga dos dados

In [61]:
# definir arquivo Pickle de entrada
#arquivo = 'dados/afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'dados/cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf-preparado.pkl.bz2'

In [62]:
df = pd.read_pickle(arquivo)
df.head()

,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,15925,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,11497,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,7515,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,7370,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [63]:
df.shape

(41146, 18)

### Expansão dos dados agregados

In [64]:
%%time
df2 = df.head(50000).copy()
#df2 = df.copy()
menor = df2['QTDE'].min()
maior = df2['QTDE'].max()
print(df2.shape[0], menor, maior, math.ceil(np.sqrt(maior / menor)))
#df2['TAM'] = df2['QTDE'].apply(lambda x: math.ceil(np.sqrt(x / menor)))
df2['TEMP'] = df2[['SEXO', 'QTDE']].apply(
    lambda x: [x[0]] * math.ceil(np.sqrt(x[1] / menor)), axis=1)
dfe = df2.explode('TEMP').drop(['TEMP'], axis=1) #.reset_index()
dfe.head(1000)

41146 1 20389 143
CPU times: user 338 ms, sys: 0 ns, total: 338 ms
Wall time: 337 ms


,SEXO,QTDE,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,,,
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
MARIA,0,20389,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0
MARCOS,1,4542,19,15,3,18,1,13,0,0,0,0,0,0,0,0,0,0


In [65]:
# usar os dados expandidos invés dos agregados
dfo = df # guardar o dataframe original
df = dfe # apontar para o dataframe expandido

### Uso dos dados originais

In [66]:
X = df.drop(['SEXO', 'QTDE'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (82709, 16) (82709,)


In [67]:
# dividir dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print("Shapes:", X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Shapes: (55415, 16) (27294, 16) (55415,) (27294,)


In [68]:
X_train.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
NOME,,,,,,,,,,,,,,,,
THIALLAN,14,1,12,12,1,9,8,20,0,0,0,0,0,0,0,0
FLAVIANE,5,14,1,9,22,1,12,6,0,0,0,0,0,0,0,0
CLAUDERINO,15,14,9,18,5,4,21,1,12,3,0,0,0,0,0,0
EDUARDO,15,4,18,1,21,4,5,0,0,0,0,0,0,0,0,0
ALEXANDRO,15,18,4,14,1,24,5,12,1,0,0,0,0,0,0,0


In [69]:
y_train.head()

NOME
THIALLAN      1
FLAVIANE      0
CLAUDERINO    1
EDUARDO       1
ALEXANDRO     1
Name: SEXO, dtype: uint8

In [70]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [71]:
# determine the number of input features
n_features = X_train.shape[1]

# define model
model = Sequential()
model.add(Dense(10, activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
model.add(Dense(8, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))

# summarize the model
model.summary()

# compile the model
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
sgd = SGD(learning_rate=0.001, momentum=0.8)
model.compile(optimizer=sgd, loss='binary_crossentropy')

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_38 (Dense)             (None, 8)                 88        
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 9         
Total params: 267
Trainable params: 267
Non-trainable params: 0
_________________________________________________________________


In [73]:
%%time

# fit the model
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0)
model

CPU times: user 2min 12s, sys: 10.8 s, total: 2min 23s
Wall time: 1min 20s


In [74]:
df2 = X_test.copy()
df2['REAL'] = y_test * 100
df2['PREV'] = model.predict(X_test) * 100
df2.head(15)

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15,REAL,PREV
NOME,,,,,,,,,,,,,,,,,,
LICIO,15,9,3,9,12,0,0,0,0,0,0,0,0,0,0,0,100,93.634201
SOLANGE,5,7,14,1,12,15,19,0,0,0,0,0,0,0,0,0,0,0.245386
BERTHA,1,8,20,18,5,2,0,0,0,0,0,0,0,0,0,0,0,1.024124
VALDENILDA,1,4,12,9,14,5,4,12,1,22,0,0,0,0,0,0,0,0.007691
JUVANETE,5,20,5,14,1,22,21,10,0,0,0,0,0,0,0,0,0,11.945590
THAISA,1,19,9,1,8,20,0,0,0,0,0,0,0,0,0,0,0,0.296915
OLINTO,15,20,14,9,12,15,0,0,0,0,0,0,0,0,0,0,100,98.835503
TRICIA,1,9,3,9,18,20,0,0,0,0,0,0,0,0,0,0,0,0.148785
SANANE,5,14,1,14,1,19,0,0,0,0,0,0,0,0,0,0,0,6.269097


In [75]:
# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Acurácia: %.2f %%' % (acc * 100))

TypeError: cannot unpack non-iterable float object

In [76]:
# save model to file
model.save('modelos/71-modelo-genero-nome-tensorflow.h5')